<a href="https://colab.research.google.com/github/yoohw0317/DAPproject/blob/main/project_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab에는 selenium을 따로 설치해야 사용할 수 있습니다.

이 selenium을 통해서 웹 크롤링을 할 수 있습니다.

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 958 kB 30.3 MB/s 
     |████████████████████████████████| 356 kB 62.1 MB/s 
     |████████████████████████████████| 138 kB 37.0 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 3.6 MB 50.4 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.dow

필요한 라이브러리들을 import해줍니다.

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--incognito')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
data = pd.DataFrame(columns = {'score', 'review'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


파싱할 호텔 트립어드바이저 전체링크를 url로 입력


In [6]:
url = 'https://www.tripadvisor.co.kr/Hotel_Review-g297884-d3813531-Reviews-Park_Hyatt_Busan-Busan.html' 
driver.get(url)
time.sleep(5)
reviews=driver.find_elements_by_css_selector("span.cvxmR")[1].text
reviews=reviews.split('(')[1].split(')')[0]
reviews=int(reviews.replace(',',''))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  after removing the cwd from sys.path.


In [7]:
i = 0
while True:
  if (reviews - i <= -5):
    break
  url_head = 'https://www.tripadvisor.co.kr/Hotel_Review-g297884-d3813531-Reviews-or' #or까지만 입력
  url_number = str(i)
  url_tail = '-Park_Hyatt_Busan-Busan.html#REVIEWS' #or 이후로 입력
  url = url_head + url_number + url_tail
  driver.get(url)

  time.sleep(5)
  try: 
    driver.find_elements_by_css_selector("div.dlJyA")[0].click()
  except:
    driver.find_elements_by_css_selector("div.dlJyA")[0].click()


  time.sleep(1)
  html_doc = driver.page_source
  soup = BeautifulSoup(html_doc)
  data_tags = soup.select('q.XllAv.H4._a span')
  data_scores = soup.select('div.emWez.F1 span.ui_bubble_rating')


  for tag, score in zip(data_tags, data_scores):
    review = str(score).split("bubble_")[2].split('"></span>')[0]
    data_insert = {'review': tag.text.strip(), 'score': int(review)/10}
    data = data.append(data_insert, ignore_index=True)
  i += 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  del sys.path[0]


In [8]:
from google.colab import files

data
data.to_excel('/home/data.xlsx')
files.download('/home/data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>